# 9


Load MNIST.

Split into 60k/10k training/test.

Train RF classifier and time the training. Evaluate Test set.

Use PCA to reduce dimensionality, explained variance of 95%.

Train new RF classifier on projected dataset, time the training. Was it faster? Evaluate on test set? How does it compare?

In [1]:
# fetch MNIST
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [2]:
X, y = mnist['data'], mnist['target']
print(f'X Shape: {X.shape}, Y Shape: {y.shape}')

X Shape: (70000, 784), Y Shape: (70000,)


In [3]:
# let's pop off a final test set (last 10,000 observations)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [4]:
print('Shape X Train: {}'.format(X_train.shape))
print('Shape Y Train: {}'.format(y_train.shape))
print('Shape X Test: {}'.format(X_test.shape))
print('Shape Y Test: {}'.format(y_test.shape))

Shape X Train: (60000, 784)
Shape Y Train: (60000,)
Shape X Test: (10000, 784)
Shape Y Test: (10000,)


In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
rfOriginalModel = RandomForestClassifier()

In [23]:
%time rfOriginalModel.fit(X_train, y_train)

CPU times: user 3.46 s, sys: 93.5 ms, total: 3.55 s
Wall time: 3.57 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
yOrigPreds = rfOriginalModel.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score

In [12]:
print('Original RF Accuracy: {0}'.format(accuracy_score(yOrigPreds, y_test)))

Original RF Accuracy: 0.9482


In [13]:
from sklearn.decomposition import PCA

In [15]:
pca = PCA(n_components=0.95)
X_PCA = pca.fit_transform(X)
# let's pop off a final test set (last 10,000 observations)
X_PCA_train, X_PCA_test, y_train, y_test = X_PCA[:60000], X_PCA[60000:], y[:60000], y[60000:]

In [19]:
X_PCA.shape

(70000, 154)

In [17]:
rfPCAModel = RandomForestClassifier()

In [22]:
%time rfPCAModel.fit(X_PCA_train, y_train)

CPU times: user 7.69 s, sys: 42.7 ms, total: 7.74 s
Wall time: 7.78 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
yPCAPreds = rfPCAModel.predict(X_PCA_test)

In [21]:
print('PCA RF Accuracy: {0}'.format(accuracy_score(yPCAPreds, y_test)))

PCA RF Accuracy: 0.8918


Interestingly, the training time was almost doubled when comparing the original dataset and the PCA dataset. Also - there was a very significant drop in accuracy score. 

Comparing my results to the author and confirmed this was the expected result of this exercise. 

PCA isn't gauranteed to speed up training and will likely always result in a degradation of accuracy.

# 10

Use t-SNE to reduct MNIST to 2-dimensions. Sample and plot (as to not clutter the viz).

Repeat this process using PCA, LLE, and MDS.

In [3]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# let's take a random sampling of 10,000 instances, 60,000 kept crashing Jupyter
m = 10000
idx = np.random.permutation(60000)[:m]

X = mnist['data'][idx]
y = mnist['target'][idx]

In [5]:
X_TSNEembedded = TSNE(n_components=2).fit_transform(X)

In [6]:
# let's pickle this since it took FOREVER to run.
import pickle
pickle.dump(X_TSNEembedded, open('X_TSNEembedded.pkl', 'wb'))

In [ ]:
plt.figure(figsize=(13,10))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, cmap="jet")
plt.axis('off')
plt.colorbar()
plt.show()

In [ ]:
X_PCA = PCA(n_components=2).fit_transform(X)